In [90]:
# 기초 전처리
import pandas as pd
import numpy as np
import os

# 크롤링
import os
from time import sleep
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

# pandas 결과값의 표현 범위 소수점 2자리수로 변경
pd.options.display.float_format = '{:.2f}'.format

# 파일 로드위한 directory 확인 및 현재 경로로 설정
a = os.getcwd()
os.chdir(a)

In [91]:
store = pd.read_csv("소상공인시장진흥공단_상가(상권)정보_서울_202106.csv", encoding='utf-8')
#df_main = pd.read_csv('url수집해야할_list.csv', encoding = 'cp949', index_col = 0)

# 데이터 크롤링

In [92]:
df = store.copy()

In [93]:
# 분류별로 수집해야 시간이 오래 안걸린다.
# 결과도 일부라도 빨리 받아 볼 수 있다.
df['상권업종대분류명'].unique()

array(['음식', '소매', '학문/교육', '부동산', '관광/여가/오락', '생활서비스', '숙박', '스포츠'],
      dtype=object)

In [94]:
# 34만개 매장
# 음식점만 일추 추출(샘플)
df = df.loc[df['상권업종대분류명'] == '음식']  

# 필요 컬럼 추출
df = df[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]

# 그 중에서도 흑석동과 상도1동 데이터만 수집 (샘플)
df = df.loc[(df['행정동명'] == '흑석동') | (df['행정동명'] == '상도1동')]

In [95]:
# 칼럼명 단순화

df.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]

## 매장별 상세페이지 url수집

In [97]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time


options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install())  # chromedriver 열기


# 검색 키워드 지정
# %20은 띄어쓰기
df['kakao_keyword'] = df['dong'] + "%20" + df['name']
df['kakao_map_url'] = ''


for i, keyword in enumerate(tqdm(df['kakao_keyword'].tolist())):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행", keyword)
    try:
        kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"
        driver.get(kakao_map_search_url)
        time.sleep(0.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li:nth-child(1) > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
    except Exception as e1: # url이 없을때
        if "li:nth-child(1)" in str(e1):  
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
                time.sleep(1)
            except Exception as e2:
                print('뽑을 url이 없습니다! error 내용 :',e2)
                df.iloc[i,-1] = np.nan
                time.sleep(0.5)
        else:
            pass
df



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\HB\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


  0%|          | 0/50 [00:00<?, ?it/s]

이번에 찾을 키워드 : 0 / 49 행 흑석동%20Bloom
이번에 찾을 키워드 : 1 / 49 행 흑석동%20노랑통닭
이번에 찾을 키워드 : 2 / 49 행 흑석동%20파티파티노래주점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=94.0.4606.61)

이번에 찾을 키워드 : 3 / 49 행 흑석동%20맥주창고
이번에 찾을 키워드 : 4 / 49 행 흑석동%20Mr.꼼장어Miss쭈꾸미
이번에 찾을 키워드 : 5 / 49 행 상도1동%20가춘
이번에 찾을 키워드 : 6 / 49 행 상도1동%20BHC치킨
이번에 찾을 키워드 : 7 / 49 행 상도1동%20대나무골
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=94.0.4606.61)

이번에 찾을 키워드 : 8 / 49 행 상도1동%20빈털털이
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_ite

In [98]:
df.to_csv('흑석동_시장.csv', encoding='cp949')

In [99]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

naver_map_name_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []

options = webdriver.ChromeOptions()
options.add_argument('headless')


# 리뷰 담을 리스트
naver_map_name_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []
naver_map_type_list = []

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(ChromeDriverManager().install())  # chromedriver 열기


# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(ChromeDriverManager().install())  # chromedriver 열기

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):

    driver.get(url)
    sub_driver.get(url+"/review/ugc")
    time.sleep(2)


    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element_by_css_selector("#_title > span._3ocDE").text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text

        # 블로그 별점 점수
        star_review_stars = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em").text

        # 블로그 별점 평가 수
        star_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(2) > a > em").text
       

        # 블로그 리뷰 텍스트 가져오기
        review_text_list = [] # 임시 선언

        
        # 네이버 지도 블로그 리뷰 탭은 동적 웹사이트의 순서가 주문하기, 메뉴보기 등의 존재 여부로 다르기 때문에 css selector가 아니라 element 찾기로 진행
        review_text_crawl_list = sub_driver.find_elements_by_class_name("_2CbII")
        
        # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
        for review_crawl_data in review_text_crawl_list:
            review_text_list.append(review_crawl_data.find_element_by_tag_name('div').text)
        
        # 그 리스트에 저장된 텍스트 (한 식당에 대한 여러 리뷰들)를 한 텍스트 덩어리로 모아(join)줍니다.
        review_text = ','.join(review_text_list)


        blog_review_list.append(review_text)

        naver_map_type_list.append(naver_map_type)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)
        naver_map_star_review_qty_list.append(star_review_qty)
        print(f"{i}행 리뷰 스크랩 완료")
    
    # 리뷰가 없는 경우
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 할당.(길이 맞추기용)
        blog_review_list.append('null')  
        naver_map_type_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')


        
driver.quit()
sub_driver.quit()


df['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
df['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점(str)
df['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수(str)
df['naver_blog_review_txt'] = blog_review_list  # 네이버 상세페이지에 나온 블로그 리뷰 텍스트들(str)
df['naver_blog_review_qty'] = blog_review_qty_list # 네이버 블로그에서 리뷰된 수(str)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\HB\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\HB\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


  0%|          | 0/27 [00:00<?, ?it/s]

0행 문제가 발생
1행 리뷰 스크랩 완료
2행 리뷰 스크랩 완료
3행 리뷰 스크랩 완료
4행 리뷰 스크랩 완료
5행 리뷰 스크랩 완료
6행 문제가 발생
7행 리뷰 스크랩 완료
8행 리뷰 스크랩 완료
9행 리뷰 스크랩 완료
10행 리뷰 스크랩 완료
11행 리뷰 스크랩 완료
12행 리뷰 스크랩 완료
13행 리뷰 스크랩 완료
14행 리뷰 스크랩 완료
15행 리뷰 스크랩 완료
16행 리뷰 스크랩 완료
17행 리뷰 스크랩 완료
18행 리뷰 스크랩 완료
19행 문제가 발생
20행 리뷰 스크랩 완료
21행 문제가 발생
22행 문제가 발생
23행 리뷰 스크랩 완료
24행 문제가 발생
25행 리뷰 스크랩 완료
26행 리뷰 스크랩 완료


In [108]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제
df = df.loc[~(df['naver_store_type'].str.contains('null'))]

In [114]:
# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 변형
df[['naver_star_point', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_blog_review_qty']].apply(pd.to_numeric)
df['naver_star_point_qty'] = df['naver_star_point_qty'].str.replace(',','').astype(int)

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 2140 to 29816
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   21 non-null     object 
 1   cate_1                 21 non-null     object 
 2   cate_2                 21 non-null     object 
 3   cate_3                 21 non-null     object 
 4   dong                   21 non-null     object 
 5   lon                    21 non-null     float64
 6   lat                    21 non-null     float64
 7   naver_keyword          21 non-null     object 
 8   naver_map_url          21 non-null     object 
 9   naver_store_type       21 non-null     object 
 10  naver_star_point       21 non-null     float64
 11  naver_star_point_qty   21 non-null     int32  
 12  naver_blog_review_txt  21 non-null     object 
 13  naver_blog_review_qty  21 non-null     int64  
dtypes: float64(3), int32(1), int64(1), object(9)
memory us

In [120]:
df

,name,cate_1,cate_2,cate_3,dong,lon,lat,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty,cate_mix
2140,노랑통닭,닭/오리요리,후라이드/양념치킨,치킨 전문점,흑석동,37.51,126.96,흑석동%20노랑통닭,https://m.place.naver.com/restaurant/38419604,"치킨,닭강정",4.45,141,서울 흑석동 중앙대 치맥 맛집. 노랑통닭 순살 3종 세트 맛있어요. 2020년 9월...,25,닭 오리요리후라이드 양념치킨치킨 전문점
2614,맥주창고,유흥주점,호프/맥주,기타 주점업,흑석동,37.51,126.96,흑석동%20맥주창고,https://m.place.naver.com/restaurant/31040500,"맥주,호프",4.50,17,흑석동 맥주창고에서 간단하게 맥주한잔!!! 퇴근하고 친구가 간단하게 한잔하자고해서 ...,5,유흥주점호프 맥주기타 주점업
3450,Mr.꼼장어Miss쭈꾸미,일식/수산물,곰장어전문,한식 음식점업,흑석동,37.51,126.96,흑석동%20Mr.꼼장어Miss쭈꾸미,https://m.place.naver.com/restaurant/12880938,"해물,생선요리",4.61,19,흑석동 중앙대 맛집 미스터꼼장어 미스쭈꾸미 얼마 전 처음으로 ! 꼼장어를 먹어보게 ...,16,일식 수산물곰장어전문한식 음식점업
6254,가춘,제과제빵떡케익,제과점,제과점업,상도1동,37.50,126.95,상도1동%20가춘,https://m.place.naver.com/restaurant/1123429741,베이커리,4.54,416,"이 날도 어김없이 걷기로 서울구경을 하던 중에 숭실대 앞을 지났고, 포스 넘치는 외...",67,제과제빵떡케익제과점제과점업
7129,BHC치킨,닭/오리요리,후라이드/양념치킨,치킨 전문점,상도1동,37.49,126.96,상도1동%20BHC치킨,https://m.place.naver.com/restaurant/37413677,"치킨,닭강정",4.48,203,bhc 포테킹 후라이드 신메뉴 가격 요기요앱에서 할인 중 ! @bhc 일요일은 거의...,8,닭 오리요리후라이드 양념치킨치킨 전문점
9083,랭스터디카페,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,상도1동,37.50,126.95,상도1동%20랭스터디카페,https://m.place.naver.com/restaurant/1612441699,장소대여,4.64,53,숭실대 스터디카페 공부하는 랭스터디카페 굿 사람들이 많이 왕래하는 곳인 숭실대 바로...,61,커피점 카페커피전문점 카페 다방비알콜 음료점업
9156,원할머니,한식,족발/보쌈전문,한식 음식점업,흑석동,37.51,126.96,흑석동%20원할머니,https://m.place.naver.com/restaurant/12765878,"족발,보쌈",4.32,142,어렷을적 햇님달님 동화책에서 본듯한 원씨?할머니께서 웃으며 반겨주시는 그 곳! 원할...,16,한식족발 보쌈전문한식 음식점업
9610,엉터리생고기숭실대점,한식,갈비/삼겹살,한식 음식점업,상도1동,37.50,126.95,상도1동%20엉터리생고기숭실대점,https://m.place.naver.com/restaurant/21033957,"육류,고기요리",4.09,81,모처럼 고기를 먹고 싶은 생각이 들어서 숭실대 주변 고기집을 찾아봤습니다 주변에 엉...,14,한식갈비 삼겹살한식 음식점업
10584,박서방네,한식,한식/백반/한정식,한식 음식점업,흑석동,37.51,126.96,흑석동%20박서방네,https://m.place.naver.com/restaurant/33622519,한식,4.11,103,사무실 건물에 식당이 있으면 참 좋을텐데 아니면 여기를 지정식당으로 하면 참 좋을텐...,7,한식한식 백반 한정식한식 음식점업
11727,가마로강정,닭/오리요리,후라이드/양념치킨,치킨 전문점,상도1동,37.50,126.95,상도1동%20가마로강정,https://m.place.naver.com/restaurant/36215164,"치킨,닭강정",4.59,335,20171128 친구랑 렌즈사러 안경집갔다가 포장마차 닭강정 발견! 먹고싶쟈냐요? ...,14,닭 오리요리후라이드 양념치킨치킨 전문점


# 코사인 유사도로 유사 매장 추천 시스템 만들기
- 추후 클러스터링할때 검증 요소 등으로 사용가능하지 않을까?

In [116]:
df['cate_mix'] = df['cate_1'] + df['cate_2'] +  df['cate_3']
df['cate_mix'] = df['cate_mix'].str.replace("/", " ")

In [117]:
# 중분류별 매장의 코사인 유사도 구하기
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [118]:
# 리뷰 텍스트 데이터 간의 텍스트 피쳐 벡터라이징
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(df['naver_blog_review_txt']) 

# 리뷰 텍스트 간의 코사인 유사도 따지기
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

In [121]:
# 추천알고리즘
# 공식 1~5의 중요성을 짬뽕시키는 공식
# * 0.003 등의 가중치를 줘서 조절합니다.

place_simi_co = (
                 + place_simi_cate * 0.3 # 공식 1. 카테고리 유사도
                 + place_simi_review * 1 # 공식 2. 리뷰 텍스트 유사도
                 + np.repeat([df['naver_blog_review_qty'].values], len(df['naver_blog_review_qty']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
                 + np.repeat([df['naver_star_point'].values], len(df['naver_star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                 + np.repeat([df['naver_star_point_qty'].values], len(df['naver_star_point_qty']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
                 )



# 아래 place_simi_co_sorted_ind 는 그냥 바로 사용하면 됩니다.
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    place_title = df[df['name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]


# 상도국수와 가장 유사한 업제 5곳 추천
find_simi_place(df, place_simi_co_sorted_ind, '상도국수')

IndexError: index 2140 is out of bounds for axis 0 with size 21